In [1]:
import pandas as pd
import geopandas as gpd
import os
file_name = "Africa_grid_10_0_40_50"
# file_name = "Africa_grid_-20_10_-15_15"
image_file_name = file_name
# file_name = "Shanghai_time_series" 
# image_file_name = "Shanghai_grid_120_30_122.25_32"
df = pd.read_csv(f"/nas/houce/Alphaearth_embedding/GEE_extracted/{file_name}/metadata/{image_file_name}_grid_cells.csv")
AEF_file_paths_all_merged = gpd.read_file("/nas/houce/Alphaearth_embedding/metadata//all_grid_cells_5x5_merged.geojson")

In [2]:
from glob import glob

finished_files = [x.split('/')[-1] for x in glob(f"/nas/houce/Alphaearth_embedding/GEE_extracted/{image_file_name}/*")]
download_zips = [x.split('/')[-1][:-4] for x in glob("/nas/houce/Alphaearth_embedding/zips/*")]
common_files = set(finished_files) & set(download_zips)
del_files = [f"/nas/houce/Alphaearth_embedding/zips/{x}.zip" for x in common_files]

for file_path in del_files:
    if os.path.exists(file_path):
        os.remove(file_path)
        print(f"已删除: {file_path}")
    else:
        print(f"文件不存在: {file_path}")

In [3]:
import numpy as np

tem = df.merge(AEF_file_paths_all_merged, on=['grid_name', 'lon_min', 'lon_max', 'lat_min', 'lat_max'], how='left')
tem = tem.drop_duplicates(subset=['grid_name', 'lon_min', 'lon_max', 'lat_min', 'lat_max']).reset_index(drop=True)
tem['file_path_len'] = tem['file_path_copied'].apply(lambda x: len(x) if isinstance(x, str) else np.nan)

In [4]:
tem[tem['file_path_len'].notna()].to_csv(f"tem.csv")
tem[tem['file_path_len'].isna()].to_csv(f"tem_na.csv")
tem[tem['file_path_len'].notna()].to_csv(f"/nas/houce/Alphaearth_embedding/GEE_extracted/{image_file_name}/metadata/tem.csv",index=None)
tem[tem['file_path_len'].isna()].to_csv(f"/nas/houce/Alphaearth_embedding/GEE_extracted/{image_file_name}/metadata/tem_na.csv",index=None)

In [5]:
duplicates = tem[tem['file_path_len'].notna()].duplicated(subset=['grid_name'], keep=False)
dup_df = tem[tem['file_path_len'].notna()][duplicates]
dup_df.drop_duplicates(subset=['grid_name'], inplace=True)
dup_dir= dup_df['file_path_len'].drop_duplicates().tolist()

In [6]:
import os
import shutil

for dir_path in dup_dir:
    if os.path.exists(dir_path) and os.path.isdir(dir_path):
        shutil.rmtree(dir_path)
        print(f"已删除目录及其内容: {dir_path}")
    elif os.path.exists(dir_path):
        os.remove(dir_path)
        print(f"已删除文件: {dir_path}")
    else:
        print(f"路径不存在: {dir_path}")